#EN3150 Assignment 03: Simple convolutional neural network to perform classification.



##1 .CNN for image classification

### Mount the google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load the Dataset

In [2]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # Progress bar to monitor the copying process

# Define the base directory and the new directories for train, validation, and test sets
base_dir = '/content/drive/My Drive/EN3150-Assignment-03-CNN/Images/'
train_dir = '/content/drive/My Drive/EN3150-Assignment-03-CNN/Images/train'
validation_dir = '/content/drive/My Drive/EN3150-Assignment-03-CNN/Images/validation'
test_dir = '/content/drive/My Drive/EN3150-Assignment-03-CNN/Images/test'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Lists to hold image data and labels
image_paths = []
labels = []

# Load images from each folder (H1, H2, H3, H5, H6)
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg'):
                # Construct full image path
                image_path = os.path.join(folder_path, filename)
                image_paths.append(image_path)
                labels.append(folder)  # Assign label based on folder name

# Convert labels to a NumPy array
labels = np.array(labels)

### Split the Dataset

In [ ]:
# Split data: 60% train, 20% validation, 20% test
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths, labels, test_size=0.4, random_state=42, stratify=labels
)
validation_paths, test_paths, validation_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

# Print the sizes of the splits
print(f'Training set: {len(train_paths)} images')
print(f'Validation set: {len(validation_paths)} images')
print(f'Test set: {len(test_paths)} images')

# Function to copy images to the relevant directories (train, validation, test)
def copy_images_to_directories(image_paths, labels, split_dir):
    # Create subdirectories for each class in the split directory (train, validation, test)
    for class_name in np.unique(labels):
        os.makedirs(os.path.join(split_dir, class_name), exist_ok=True)

    # Use tqdm to show progress while copying files
    for image_path, label in tqdm(zip(image_paths, labels), total=len(image_paths)):
        # Determine the class directory for the label
        class_dir = os.path.join(split_dir, label)
        # Copy the image to the appropriate class directory
        shutil.copy(image_path, class_dir)

# Copy images to the train, validation, and test directories
print("Copying images to the train directory...")
copy_images_to_directories(train_paths, train_labels, train_dir)

print("Copying images to the validation directory...")
copy_images_to_directories(validation_paths, validation_labels, validation_dir)

print("Copying images to the test directory...")
copy_images_to_directories(test_paths, test_labels, test_dir)

print("Images have been copied to train, validation, and test directories.")


Training set: 5468 images
Validation set: 1823 images
Test set: 1823 images
Copying images to the train directory...


100%|██████████| 5468/5468 [14:18<00:00,  6.37it/s]


Copying images to the validation directory...


100%|██████████| 1823/1823 [04:19<00:00,  7.03it/s]


Copying images to the test directory...


100%|██████████| 1823/1823 [04:21<00:00,  6.98it/s]

Images have been copied to train, validation, and test directories.


### Build the CNN Model with modifications

In [8]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Define the CNN architecture
def build_cnn(input_shape,x1, m1, x2, m2, x3, x4, m3, d, num_classes):
    model = models.Sequential(name="Custom_CNN_Model")

    # Use Input layer to define the input shape
    model.add(layers.Input(shape=input_shape, name='Input'))

    # First Convolutional Layer
    model.add(layers.Conv2D(x1, (m1, m1), activation='relu', name='Conv2D_1'))
    model.add(layers.MaxPooling2D((2, 2), name='MaxPooling2D_1'))

    # Second Convolutional Layer
    model.add(layers.Conv2D(x2, (m2, m2), activation='relu', name='Conv2D_2'))
    model.add(layers.MaxPooling2D((2, 2), name='MaxPooling2D_2'))

    # Third Convolutional Layer
    model.add(layers.Conv2D(x4, (m3, m3), activation='relu', name='Conv2D_3'))
    model.add(layers.MaxPooling2D((2, 2), name='MaxPooling2D_3'))

    # Fourth Convolutional layer
    model.add(layers.Conv2D(x4, (m3, m3), activation='relu', name='Conv2D_4'))
    model.add(layers.MaxPooling2D((2, 2), name='MaxPooling2D_4'))

    # Flatten the output
    model.add(layers.Flatten(name='Flatten'))

    # Fully Connected Layer
    model.add(layers.Dense(x3, activation='relu', name='Dense_1'))
    model.add(layers.Dropout(d, name='Dropout_1'))  # Dropout layer to reduce overfitting

    # Output Layer
    model.add(layers.Dense(num_classes, activation='softmax', name='Output'))

    return model

# Build and compile the CNN model
input_shape = (224,224, 3)
cnn_model = build_cnn(input_shape,x1=32, m1=3, x2=64, m2=3, x3=128, x4=64, m3=3, d=0.5, num_classes=5)

# Custom learning rate
custom_lr_optimizer = Adam(learning_rate=0.0001)

# Using the optimizer in the model
cnn_model.compile(optimizer=custom_lr_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
cnn_model.summary()

Model: "Custom_CNN_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv2D_1 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 MaxPooling2D_1 (MaxPooling  (None, 111, 111, 32)      0         
 2D)                                                             
                                                                 
 Conv2D_2 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 MaxPooling2D_2 (MaxPooling  (None, 54, 54, 64)        0         
 2D)                                                             
                                                                 
 Conv2D_3 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 MaxPooling2D_3 (MaxPooling  (None, 26, 26, 64)   

### Train the Model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
img_size = (224, 224)  # Resize images to 150x150
batch_size = 32

# Create ImageDataGenerators for training, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalize images to [0, 1]
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use categorical since labels are one-hot encoded
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No shuffling for validation data
)

# Train the model
history = cnn_model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    verbose=1
)

save_path = '/content/drive/My Drive/models/cnn_model_1'
cnn_model.save(save_path, save_format='tf')

Found 5468 images belonging to 5 classes.
Found 1823 images belonging to 5 classes.
Epoch 1/20
171/171 [==============================] - 1283s 8s/step - loss: 1.2660 - accuracy: 0.5110 - val_loss: 1.2145 - val_accuracy: 0.5315
Epoch 2/20
171/171 [==============================] - 378s 2s/step - loss: 1.0909 - accuracy: 0.5772 - val_loss: 0.9759 - val_accuracy: 0.6072
Epoch 3/20
171/171 [==============================] - 374s 2s/step - loss: 1.0228 - accuracy: 0.5925 - val_loss: 0.9541 - val_accuracy: 0.6089
Epoch 4/20
171/171 [==============================] - 374s 2s/step - loss: 1.0041 - accuracy: 0.6020 - val_loss: 0.9190 - val_accuracy: 0.6330
Epoch 5/20
171/171 [==============================] - 374s 2s/step - loss: 0.9711 - accuracy: 0.6165 - val_loss: 0.9133 - val_accuracy: 0.6413
Epoch 6/20
171/171 [==============================] - 375s 2s/step - loss: 0.9763 - accuracy: 0.6117 - val_loss: 0.9306 - val_accuracy: 0.6341
Epoch 7/20
170/171 [============================>.] - ETA

### Evaluate the model

In [ ]:
# Create a data generator for the test dataset
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No shuffling to maintain order for evaluation
)

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = cnn_model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_accuracy:.2f}")

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Get true labels and predictions
true_labels = test_generator.classes
predictions = cnn_model.predict(test_generator, verbose=1)
predicted_labels = np.argmax(predictions, axis=1)

# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Classification report
report = classification_report(true_labels, predicted_labels, target_names=class_labels)
print("Classification Report:")
print(report)


In [ ]:
train_accuracy = history.history['accuracy'][-1]
val_accuracy = history.history['val_accuracy'][-1]

print(f"Training Accuracy: {train_accuracy:.2f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")


### Plot training and validation loss

In [ ]:
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Learning rates to experiment with
learning_rates = [0.0001, 0.001, 0.01, 0.1]

# Dictionary to store training histories for each learning rate
histories = {}

for lr in learning_rates:
    print(f"Training with learning rate: {lr}")

    # Build and compile the model with the given learning rate
    cnn_model = build_cnn(input_shape=(224, 224, 3), x1=32, m1=3, x2=64, m2=3, x3=128, x4=64, m3=3, d=0.5, num_classes=5)
    cnn_model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model and store history
    history = cnn_model.fit(
        train_generator,
        epochs=10,  # Use a smaller number of epochs for quicker evaluation
        validation_data=validation_generator,
        verbose=1
    )

    histories[lr] = history
